In [25]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

import pandas as pd
import numpy
import tensorflow as tf
import matplotlib.pyplot as plt 
import seaborn as sns 

# seed 값 설정
seed=0
numpy.random.seed(3)
tf.random.set_seed(3)

In [26]:
# 데이터 불러오기
df = pd.read_csv('data/TSLA.csv')
df.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-07-01,5.000,5.184,4.054,4.392,4.392,41094000
1,2010-07-02,4.600,4.620,3.742,3.840,3.840,25699000
2,2010-07-06,4.000,4.000,3.166,3.222,3.222,34334500
3,2010-07-07,3.280,3.326,2.996,3.160,3.160,34608500
4,2010-07-08,3.228,3.504,3.114,3.492,3.492,38557000
5,2010-07-09,3.516,3.580,3.310,3.480,3.480,20253000
6,2010-07-12,3.590,3.614,3.400,3.410,3.410,11012500
7,2010-07-13,3.478,3.728,3.380,3.628,3.628,13400500
8,2010-07-14,3.588,4.030,3.552,3.968,3.968,20976000
9,2010-07-15,3.988,4.300,3.800,3.978,3.978,18699000


In [27]:
df.info()
#날짜는 object 타입 Volume은 int 타입 나머진 모두 float 타입
#예측할 값은 Close 종가

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2843 entries, 0 to 2842
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       2843 non-null   object 
 1   Open       2843 non-null   float64
 2   High       2843 non-null   float64
 3   Low        2843 non-null   float64
 4   Close      2843 non-null   float64
 5   Adj Close  2843 non-null   float64
 6   Volume     2843 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 155.6+ KB


In [28]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,2843.000000,2843.000000,2843.000000,2843.000000,2843.000000,2.843000e+03
mean,105.868475,108.031370,103.555733,105.924597,105.924597,3.141524e+07
std,188.738974,192.483055,184.638617,188.836358,188.836358,2.841880e+07
min,3.228000,3.326000,2.996000,3.160000,3.160000,5.925000e+05
25%,10.698000,11.026000,10.420000,10.727000,10.727000,1.251050e+07
50%,45.874001,46.493999,45.102001,45.916000,45.916000,2.481500e+07
75%,65.021000,66.251999,64.015001,65.275002,65.275002,4.012025e+07
max,891.380005,900.400024,871.599976,883.090027,883.090027,3.046940e+08


In [29]:
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].apply(lambda x : x.year)
df['Month'] = df['Date'].apply(lambda x : x.month)
df['Day'] = df['Date'].apply(lambda x : x.day)
df = df.drop('Date', axis=1)

In [30]:
df

,Open,High,Low,Close,Adj Close,Volume,Year,Month,Day
0,5.000000,5.184000,4.054000,4.392000,4.392000,41094000,2010,7,1
1,4.600000,4.620000,3.742000,3.840000,3.840000,25699000,2010,7,2
2,4.000000,4.000000,3.166000,3.222000,3.222000,34334500,2010,7,6
3,3.280000,3.326000,2.996000,3.160000,3.160000,34608500,2010,7,7
4,3.228000,3.504000,3.114000,3.492000,3.492000,38557000,2010,7,8
...,...,...,...,...,...,...,...,...,...
2838,796.210022,796.380005,780.909973,785.489990,785.489990,16711100,2021,10,8
2839,787.650024,801.239990,785.500000,791.940002,791.940002,14200300,2021,10,11
2840,800.929993,812.320007,796.570007,805.719971,805.719971,22020000,2021,10,12
2841,810.469971,815.409973,805.780029,811.080017,811.080017,14120100,2021,10,13


In [31]:
df.corr()

,Open,High,Low,Close,Adj Close,Volume,Year,Month,Day
Open,1.000000,0.999763,0.999691,0.999444,0.999444,0.144874,0.639221,-0.019727,-0.004057
High,0.999763,1.000000,0.999606,0.999771,0.999771,0.150014,0.639921,-0.020241,-0.003458
Low,0.999691,0.999606,1.000000,0.999684,0.999684,0.139580,0.638684,-0.018690,-0.003202
Close,0.999444,0.999771,0.999684,1.000000,1.000000,0.145800,0.639262,-0.019041,-0.002870
Adj Close,0.999444,0.999771,0.999684,1.000000,1.000000,0.145800,0.639262,-0.019041,-0.002870
Volume,0.144874,0.150014,0.139580,0.145800,0.145800,1.000000,0.474363,-0.077593,-0.048336
Year,0.639221,0.639921,0.638684,0.639262,0.639262,0.474363,1.000000,-0.108436,-0.002577
Month,-0.019727,-0.020241,-0.018690,-0.019041,-0.019041,-0.077593,-0.108436,1.000000,-0.002661
Day,-0.004057,-0.003458,-0.003202,-0.002870,-0.002870,-0.048336,-0.002577,-0.002661,1.000000


In [32]:
X = df.drop(['Close', 'Adj Close'], axis=1)
Y = df['Adj Close']

In [33]:
#데이터를 예측하기 쉽게 0과 1사이의 값으로 바꿔줌
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [34]:
#훈련셋과 테스트셋 분리
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=seed)

In [37]:
#모델 프레임 생성
#Linear Regression이므로 sigmoid 사용
model = Sequential()
model.add(Dense(32, input_dim = 7, activation='relu'))
model.add(Dense(1, activation='linear'))

In [40]:
model.compile(loss='mean_squared_error',
              optimizer='adam')

model.fit(X_train, Y_train, epochs=100, validation_split=0.2,batch_size = 10)

Train on 1592 samples, validate on 398 samples
Epoch 1/50
1592/1592 [==============================] - 1s 315us/step - loss: 24.5073 - val_loss: 25.5857
Epoch 2/50
1592/1592 [==============================] - 0s 266us/step - loss: 22.8796 - val_loss: 23.8678
Epoch 3/50
1592/1592 [==============================] - 0s 251us/step - loss: 21.4206 - val_loss: 21.4780
Epoch 4/50
1592/1592 [==============================] - 0s 301us/step - loss: 20.3045 - val_loss: 20.3681
Epoch 5/50
1592/1592 [==============================] - 1s 322us/step - loss: 19.6310 - val_loss: 20.4299
Epoch 6/50
1592/1592 [==============================] - 0s 304us/step - loss: 19.1806 - val_loss: 19.5183
Epoch 7/50
1592/1592 [==============================] - 0s 273us/step - loss: 18.7227 - val_loss: 19.1561
Epoch 8/50
1592/1592 [==============================] - 0s 289us/step - loss: 18.4395 - val_loss: 20.9930
Epoch 9/50
1592/1592 [==============================] - 0s 275us/step - loss: 18.5930 - val_loss: 20.7587

In [39]:
test_loss = model.evaluate(X_test, Y_test, verbose=0)

print("Test Loss: {:.5f}".format(test_loss))

Test Loss: 49.01756
